In [35]:
import pandas as pd
import os


In [147]:
import os
import pandas as pd

# Directory where CSV files are stored
csv_directory = '/Users/wickywonka/CMU/coursework/17735/project/17735-project/ExtractedData'

# Lists to hold data from each file
logon_data = []
logoff_data = []

# Iterate over each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Full path to the CSV file
        csv_path = os.path.join(csv_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_path, names=['User', 'Timestamp', 'PC', 'ActivityType', 'Action'])

        # Convert Timestamp to datetime
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')

        # Extract time and hour from Timestamp
        df['Time'] = df['Timestamp'].dt.time
        df['Hour'] = df['Timestamp'].dt.hour

        # Append DataFrame to the list by action
        logon_data.append(df[df['Action'] == 'Logon'])
        logoff_data.append(df[df['Action'] == 'Logoff'])

# Concatenate all logon and logoff data into two separate DataFrames
logon_df = pd.concat(logon_data)
logoff_df = pd.concat(logoff_data)

# Group by 'User' and aggregate times for logon
df_user_logon_stats = logon_df.groupby('User')['Time'].agg(['min', 'max']).reset_index()
df_logon_mode = logon_df.groupby('User')['Time'].agg(lambda x: x.value_counts().index[0]).reset_index()
df_logon_mean = logon_df.groupby('User')['Hour'].mean().reset_index()

# Convert mean hour to int and then to time
df_logon_mean['Hour'] = df_logon_mean['Hour'].astype(int)
df_logon_mean['Hour'] = pd.to_datetime(df_logon_mean['Hour'], format='%H').dt.time

# Add mode and mean to the logon stats DataFrame
df_user_logon_stats['mode'] = df_logon_mode['Time']
df_user_logon_stats['mean'] = df_logon_mean['Hour']

# Group by 'User' and aggregate times for logoff
df_user_logoff_stats = logoff_df.groupby('User')['Time'].agg(['min', 'max']).reset_index()
df_logoff_mode = logoff_df.groupby('User')['Time'].agg(lambda x: x.value_counts().index[0]).reset_index()
df_logoff_mean = logoff_df.groupby('User')['Hour'].mean().reset_index()

# Convert mean hour to int and then to time
df_logoff_mean['Hour'] = df_logoff_mean['Hour'].astype(int)
df_logoff_mean['Hour'] = pd.to_datetime(df_logoff_mean['Hour'], format='%H').dt.time

# Add mode and mean to the logoff stats DataFrame
df_user_logoff_stats['mode'] = df_logoff_mode['Time']
df_user_logoff_stats['mean'] = df_logoff_mean['Hour']


In [148]:
print(df_user_logon_stats)
print(df_user_logoff_stats)

             User       min       max      mode      mean
0    DTAA/AAA0371  07:35:01  13:46:57  07:54:21  09:00:00
1    DTAA/AAC0344  07:50:03  08:09:53  08:06:07  07:00:00
2    DTAA/AAC0599  07:50:03  08:09:58  07:56:28  07:00:00
3    DTAA/AAH0734  08:20:05  08:39:58  08:26:55  08:00:00
4    DTAA/AAK0658  08:20:00  08:39:54  08:22:33  08:00:00
..            ...       ...       ...       ...       ...
995  DTAA/ZGH0528  08:35:06  14:54:12  08:36:42  10:00:00
996  DTAA/ZKE0662  07:20:03  13:36:39  07:23:43  09:00:00
997  DTAA/ZKH0388  06:35:00  16:35:13  06:47:44  08:00:00
998  DTAA/ZKN0548  08:20:05  08:39:46  08:25:35  08:00:00
999  DTAA/ZRR0705  09:05:02  09:24:54  09:09:14  09:00:00

[1000 rows x 5 columns]
             User       min       max      mode      mean
0    DTAA/AAA0371  08:33:18  14:54:58  14:48:22  13:00:00
1    DTAA/AAC0344  15:55:00  16:09:58  15:55:10  15:00:00
2    DTAA/AAC0599  15:55:01  16:09:53  16:04:27  15:00:00
3    DTAA/AAH0734  17:25:00  17:39:59  17:34:26

In [149]:
df_user_logon_stats


,User,min,max,mode,mean
0,DTAA/AAA0371,07:35:01,13:46:57,07:54:21,09:00:00
1,DTAA/AAC0344,07:50:03,08:09:53,08:06:07,07:00:00
2,DTAA/AAC0599,07:50:03,08:09:58,07:56:28,07:00:00
3,DTAA/AAH0734,08:20:05,08:39:58,08:26:55,08:00:00
4,DTAA/AAK0658,08:20:00,08:39:54,08:22:33,08:00:00
...,...,...,...,...,...
995,DTAA/ZGH0528,08:35:06,14:54:12,08:36:42,10:00:00
996,DTAA/ZKE0662,07:20:03,13:36:39,07:23:43,09:00:00
997,DTAA/ZKH0388,06:35:00,16:35:13,06:47:44,08:00:00
998,DTAA/ZKN0548,08:20:05,08:39:46,08:25:35,08:00:00


In [150]:
df_user_logoff_stats

,User,min,max,mode,mean
0,DTAA/AAA0371,08:33:18,14:54:58,14:48:22,13:00:00
1,DTAA/AAC0344,15:55:00,16:09:58,15:55:10,15:00:00
2,DTAA/AAC0599,15:55:01,16:09:53,16:04:27,15:00:00
3,DTAA/AAH0734,17:25:00,17:39:59,17:34:26,17:00:00
4,DTAA/AAK0658,16:25:05,16:39:59,16:38:32,16:00:00
...,...,...,...,...,...
995,DTAA/ZGH0528,17:40:01,17:54:58,17:50:29,17:00:00
996,DTAA/ZKE0662,19:25:02,19:39:59,19:37:22,19:00:00
997,DTAA/ZKH0388,07:34:06,17:54:59,17:47:13,16:00:00
998,DTAA/ZKN0548,18:25:01,18:39:54,18:33:30,18:00:00


In [151]:
df_logon_mean.head()

,User,Hour
0,DTAA/AAA0371,09:00:00
1,DTAA/AAC0344,07:00:00
2,DTAA/AAC0599,07:00:00
3,DTAA/AAH0734,08:00:00
4,DTAA/AAK0658,08:00:00


In [152]:
df_logon_mode.head()

,User,Time
0,DTAA/AAA0371,07:54:21
1,DTAA/AAC0344,08:06:07
2,DTAA/AAC0599,07:56:28
3,DTAA/AAH0734,08:26:55
4,DTAA/AAK0658,08:22:33


In [153]:
df_logoff_mean.head()

,User,Hour
0,DTAA/AAA0371,13:00:00
1,DTAA/AAC0344,15:00:00
2,DTAA/AAC0599,15:00:00
3,DTAA/AAH0734,17:00:00
4,DTAA/AAK0658,16:00:00


In [154]:
df_logoff_mode.head()

,User,Time
0,DTAA/AAA0371,14:48:22
1,DTAA/AAC0344,15:55:10
2,DTAA/AAC0599,16:04:27
3,DTAA/AAH0734,17:34:26
4,DTAA/AAK0658,16:38:32


In [161]:
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pickle
import numpy as np
import pandas as pd 

def dtt2timestamp(dtt):
  time_in_sec = (dtt.hour*60 + dtt.minute) * 60 + dtt.second
  return time_in_sec

df_user_logon_stats_sec = df_user_logon_stats

time_columns = ['min', 'max', 'mode', 'mean']
for col in time_columns:
    df_user_logon_stats_sec[col] = pd.to_datetime(df_user_logon_stats[col], format='%H:%M:%S').dt.time


min_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['min']]
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mean']]

df_user_logon_stats_sec['min_ts'] = min_ts
df_user_logon_stats_sec['max_ts'] = max_ts
df_user_logon_stats_sec['mode_ts'] = mode_ts
df_user_logon_stats_sec['mean_ts'] = mean_ts


df_user_logon_stats_sec.drop(['min','max','mode','mean'], axis=1)

df_user_logoff_stats_sec = df_user_logoff_stats

for col in time_columns:
    df_user_logoff_stats_sec[col] = pd.to_datetime(df_user_logoff_stats[col], format='%H:%M:%S').dt.time


min_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['min']] 
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mean']]


df_user_logoff_stats_sec['min_ts'] = min_ts
df_user_logoff_stats_sec['max_ts'] = max_ts
df_user_logoff_stats_sec['mode_ts'] = mode_ts
df_user_logoff_stats_sec['mean_ts'] = mean_ts


df_user_logoff_stats_sec.drop(['min', 'max','mode','mean'], axis=1)


df_log_on_off_stats = pd.DataFrame()

df_log_on_off_stats['User'] = df_user_logon_stats_sec['User']
df_log_on_off_stats['on_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['on_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['on_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['on_mean_ts'] = df_user_logon_stats_sec['mean_ts']
df_log_on_off_stats['off_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['off_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['off_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['off_mean_ts'] = df_user_logon_stats_sec['mean_ts']


#df_log_on_off_stats.dtypes

log_stats = df_log_on_off_stats.drop(['User'], axis=1)
log_stats_array = np.array(log_stats)  # Changed from np.matrix to np.array
print(log_stats_array)



#df_log_on_off_stats.columns


#logon
forest = IsolationForest(bootstrap=False, contamination= 0.1 , max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=1, random_state=0,
        verbose=0)
forest.fit(log_stats_array)
log_ascore = forest.decision_function(log_stats_array)
log_ascore[:10]


# counting the values
df22 = pd.Series(forest.predict(log_stats_array))
df22 = df22.map({1:0, -1:1})
print(df22.value_counts())


# with open('../pkl/log_stats_matrix.pkl','wb') as file :
#   pickle.dump(forest, file)


df_user_log_result = pd.DataFrame()
df_user_log_result['User'] = df_user_logoff_stats_sec['User']
df_user_log_result['ascore'] = log_ascore
# df_user_log_result.to_csv('user_log_result.csv')
print(df_user_log_result)


outliers = df_user_log_result.loc[df_user_log_result['ascore'] < 0]
print(outliers)



[[27301 49617 28461 ... 49617 28461 32400]
 [28203 29393 29167 ... 29393 29167 25200]
 [28203 29398 28588 ... 29398 28588 25200]
 ...
 [23700 59713 24464 ... 59713 24464 28800]
 [30005 31186 30335 ... 31186 30335 28800]
 [32702 33894 32954 ... 33894 32954 32400]]
0    900
1    100
Name: count, dtype: int64
             User    ascore
0    DTAA/AAA0371  0.146561
1    DTAA/AAC0344  0.171652
2    DTAA/AAC0599  0.184828
3    DTAA/AAH0734  0.150257
4    DTAA/AAK0658  0.146880
..            ...       ...
995  DTAA/ZGH0528  0.097737
996  DTAA/ZKE0662  0.112834
997  DTAA/ZKH0388  0.002084
998  DTAA/ZKN0548  0.147484
999  DTAA/ZRR0705  0.017965

[1000 rows x 2 columns]
             User    ascore
13   DTAA/ABS0726 -0.005567
14   DTAA/ACD0647 -0.057754
24   DTAA/AFF0760 -0.002316
33   DTAA/AGW0182 -0.001182
45   DTAA/AJC0399 -0.087825
..            ...       ...
954  DTAA/WCN0570 -0.046487
961  DTAA/WJG0153 -0.038810
970  DTAA/WTA0867 -0.034294
972  DTAA/XCB0445 -0.043429
975  DTAA/XIN0791 -0.02

In [131]:
# Lists to hold data from each file
connect_data = []
disconnect_data = []

# Iterate over each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Full path to the CSV file
        csv_path = os.path.join(csv_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_path, names=['User', 'Timestamp', 'PC', 'ActivityType', 'Action'])

        # Convert Timestamp to datetime
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')

        # Filter DataFrame by action and append to the respective list
        connect_data.append(df[df['Action'] == 'Connect'])
        disconnect_data.append(df[df['Action'] == 'Disconnect'])

# Concatenate all connect and disconnect data into two separate DataFrames
connect_df = pd.concat(connect_data)
disconnect_df = pd.concat(disconnect_data)

# Calculate means for connect and disconnect
connect_means = connect_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())
disconnect_means = disconnect_df.groupby('User')['Timestamp'].apply(lambda x: x.mean().time())

# Calculate modes for connect and disconnect
# We use scipy's mode function which returns the first mode in case of multimodal data
connect_modes = connect_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])
disconnect_modes = disconnect_df.groupby('User')['Timestamp'].apply(lambda x: x.dt.time.mode()[0])

# Convert Series to DataFrame for CSV output
connect_means_df = connect_means.reset_index().rename(columns={'Timestamp': 'Connect_mean_time'})
connect_modes_df = connect_modes.reset_index().rename(columns={'Timestamp': 'Connect_mode_time'})

disconnect_means_df = disconnect_means.reset_index().rename(columns={'Timestamp': 'Disconnect_mean_time'})
disconnect_modes_df = disconnect_modes.reset_index().rename(columns={'Timestamp': 'Disconnect_mode_time'})

# Save the DataFrames to CSV files
# connect_means_df.to_csv('connect_means.csv', index=False)
# connect_modes_df.to_csv('connect_modes.csv', index=False)
# disconnect_means_df.to_csv('disconnect_means.csv', index=False)
# disconnect_modes_df.to_csv('disconnect_modes.csv', index=False)

# These variables hold the respective statistics for later use in your code:
# connect_means_df, connect_modes_df, disconnect_means_df, disconnect_modes_df

In [132]:
connect_means_df.head()

,User,Connect_mean_time
0,DTAA/ABB0272,19:20:46.508823
1,DTAA/ABS0726,00:06:48.806451
2,DTAA/ACH0803,14:59:06.423791
3,DTAA/AFF0760,03:13:46.333333
4,DTAA/AFH0331,10:56:46.738007


In [133]:
connect_modes_df.head()

,User,Connect_mode_time
0,DTAA/ABB0272,15:47:45
1,DTAA/ABS0726,01:01:07
2,DTAA/ACH0803,16:05:54
3,DTAA/AFF0760,00:57:03
4,DTAA/AFH0331,07:57:22


In [134]:
disconnect_means_df.head()

,User,Disconnect_mean_time
0,DTAA/ABB0272,22:43:29.416393
1,DTAA/ABS0726,00:36:46.290322
2,DTAA/ACH0803,06:03:02.383333
3,DTAA/AFF0760,03:44:57.333333
4,DTAA/AFH0331,09:26:35.647541


In [135]:
disconnect_modes_df.head()

,User,Disconnect_mode_time
0,DTAA/ABB0272,14:35:34
1,DTAA/ABS0726,01:01:46
2,DTAA/ACH0803,16:43:48
3,DTAA/AFF0760,01:02:11
4,DTAA/AFH0331,17:47:54


In [136]:
df_all = pd.DataFrame()

# Read all CSV files and concatenate into a single DataFrame
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_directory, filename)
        df_temp = pd.read_csv(file_path, header=None)  # Assuming the CSV has no header
        df_temp.columns = ['User', 'Timestamp', 'PC', 'Activity', 'Action']  # Assign column names
        df_all = pd.concat([df_all, df_temp])

# Ensure 'Timestamp' is a datetime object
df_all['Timestamp'] = pd.to_datetime(df_all['Timestamp'], format='%m/%d/%Y %H:%M:%S')

# Filter out 'Logon' and 'Logoff' activities into separate DataFrames
df_logon = df_all[df_all['Action'] == 'Logon']
df_logoff = df_all[df_all['Action'] == 'Logoff']

# Define a function to perform the aggregation and transformation
def aggregate_user_pc_log_activities(df_activity):
    # Aggregate data to count the total number of activities per user per PC
    df_user_pc = df_activity.groupby(['User', 'PC']).size().reset_index(name='pc_activities_per_user_total')

    # Calculate the unique count of PCs per user
    df_user_pc['unique_pc_count'] = df_user_pc.groupby('User')['PC'].transform('nunique')

    # Keep only the 'User' and 'unique_pc_count' columns and drop duplicates
    df_user_pc = df_user_pc[['User', 'unique_pc_count']].drop_duplicates()

    return df_user_pc

# Apply the function to both logon and logoff DataFrames
df_user_pc_logon = aggregate_user_pc_log_activities(df_logon)
df_user_pc_logoff = aggregate_user_pc_log_activities(df_logoff)

# Optionally, save these dataframes to CSV
# df_user_pc_logon.to_csv('user_pc_logon.csv', index=False)
# df_user_pc_logoff.to_csv('user_pc_logoff.csv', index=False)

In [137]:
df_user_pc_logon

,User,unique_pc_count
0,DTAA/AAA0371,16
16,DTAA/AAC0344,1
17,DTAA/AAC0599,1
18,DTAA/AAH0734,1
19,DTAA/AAK0658,1
...,...,...
22244,DTAA/ZGH0528,1
22245,DTAA/ZKE0662,1
22246,DTAA/ZKH0388,44
22290,DTAA/ZKN0548,1


In [138]:
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pickle
import numpy as np

In [139]:
# Let's assume df_user_pc_logon and df_user_pc_logoff are obtained from the previous code
# We will use df_user_pc_logon for this example

# Prepare the data for the Isolation Forest
# Reshape the 'unique_pc_count' column to a 2D array
pc_counts = np.array(df_user_pc_logon['unique_pc_count']).reshape(-1, 1)

# Initialize the Isolation Forest model
forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
                         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
                         verbose=0)

# Fit the model
forest.fit(pc_counts)

# Serialize the model using pickle
with open('user_pc_ct_isolation_forest.pkl', 'wb') as file:
    pickle.dump(forest, file)

# Get the anomaly scores (the lower, the more abnormal)
anomaly_scores = forest.decision_function(pc_counts)

# Create a DataFrame to hold the results
results = pd.DataFrame()
results['user'] = df_user_pc_logon['User']
results['unique_pc_count'] = df_user_pc_logon['unique_pc_count']
results['anomaly_score'] = anomaly_scores

# Identify potential outliers
# Outliers are defined as observations with an anomaly score less than 0
outliers = results.loc[results['anomaly_score'] < 0]

# Display the results
print(results.head(10))  # Show the first 10 results
print(outliers)          # Show the outliers

             user  unique_pc_count  anomaly_score
0    DTAA/AAA0371               16       0.022007
16   DTAA/AAC0344                1       0.174553
17   DTAA/AAC0599                1       0.174553
18   DTAA/AAH0734                1       0.174553
19   DTAA/AAK0658                1       0.174553
20   DTAA/AAK0924                1       0.174553
21   DTAA/AAR0508                1       0.174553
22   DTAA/AAS0709               19       0.045670
41   DTAA/ABB0272               72      -0.046046
113  DTAA/ABD0833                1       0.174553
               user  unique_pc_count  anomaly_score
41     DTAA/ABB0272               72      -0.046046
117    DTAA/ABS0726               11      -0.011135
186    DTAA/AFG0122               74      -0.085847
320    DTAA/AGW0182               65      -0.014885
557    DTAA/AJN0804               71      -0.036564
...             ...              ...            ...
21634  DTAA/VFP0669               67      -0.019362
21719  DTAA/VMM0309               

In [144]:
print(df_user_logon_stats)

             User    min    max   mode   mean
0    DTAA/AAA0371  27301  49617  28461  32400
1    DTAA/AAC0344  28203  29393  29167  25200
2    DTAA/AAC0599  28203  29398  28588  25200
3    DTAA/AAH0734  30005  31198  30415  28800
4    DTAA/AAK0658  30000  31194  30153  28800
..            ...    ...    ...    ...    ...
995  DTAA/ZGH0528  30906  53652  31002  36000
996  DTAA/ZKE0662  26403  48999  26623  32400
997  DTAA/ZKH0388  23700  59713  24464  28800
998  DTAA/ZKN0548  30005  31186  30335  28800
999  DTAA/ZRR0705  32702  33894  32954  32400

[1000 rows x 5 columns]


ValueError: time data "27301" doesn't match format "%H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.